In [ ]:
library('purrr')
library('tidyverse')

theme_set(theme_gray(base_size = 16))

In [ ]:
tosses = c(rep('heads', 140), rep('tails', 110))

tosses[1:5]

In [ ]:
length(tosses)

In [ ]:
bootstrap_resample = function(tosses, expected = 125) {
    
    result = sample(tosses, replace = TRUE)
    
    result_df = data.frame(outcome = result) |>
        count(outcome) |>
        filter(outcome == 'heads') |>
        mutate(difference_from_expected = n - expected)
    
    return(result_df)
}

n_resamplings = 1000

samples_df = map_df(1:n_resamplings, ~bootstrap_resample(tosses), .id = 'sample_id')

samples_df |> head()

In [ ]:
get_percentiles = function(x, q=c(0.025, 0.5, 0.975)) {
    tibble(value = quantile(x, q), percentile = q)
}

percentile_df = samples_df |>
    reframe(get_percentiles(difference_from_expected, q=c(0.025, 0.975))) |>
    tidyr::pivot_wider(names_from = percentile)
    
percentile_df

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 4)

p = ggplot() + 
    geom_rect(data = percentile_df, 
              mapping = aes(xmin = `0.025`, xmax = `0.975`, ymin = -Inf, ymax = Inf),
              alpha = 0.35, fill = 'MediumAquaMarine') +
    geom_histogram(data = samples_df, mapping = aes(x = difference_from_expected)) +
    geom_vline(xintercept = 0, color = 'firebrick', linewidth = 1)

p